Pre Processing the data

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/ISL/new_video_coordinates_21.csv')

label_encoder = LabelEncoder()
df['Word'] = label_encoder.fit_transform(df['Word'])

df['Video'] = pd.factorize(df['Video'])[0]
df['frame'] = pd.factorize(df['Frame'])[0]

sequences = df.groupby(['Word', 'Video', 'Frame'])[['Landmark', 'X', 'Y', 'Z']].apply(lambda x: x.values.tolist()).tolist()
y = df.groupby(['Word', 'Video', 'Frame'])['Word'].first().tolist()

X = [np.array(seq) for seq in sequences]
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_padded = pad_sequences(X_train, padding='post', dtype='float32', truncating='post')

KeyboardInterrupt: 

Designing model

In [ ]:
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_padded.shape[1],X_train_padded.shape[2])),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(np.max(y) + 1, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Training the model

In [ ]:
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1858/1858 [==============================] - 87s 45ms/step - loss: 2.1888 - accuracy: 0.3018 - val_loss: 1.3088 - val_accuracy: 0.5593
Epoch 2/10
1858/1858 [==============================] - 85s 46ms/step - loss: 1.1899 - accuracy: 0.5999 - val_loss: 1.1339 - val_accuracy: 0.6246
Epoch 3/10
1858/1858 [==============================] - 80s 43ms/step - loss: 0.9760 - accuracy: 0.6701 - val_loss: 0.8598 - val_accuracy: 0.7133
Epoch 4/10
1858/1858 [==============================] - 79s 43ms/step - loss: 0.8510 - accuracy: 0.7109 - val_loss: 0.7998 - val_accuracy: 0.7314
Epoch 5/10
1858/1858 [==============================] - 83s 45ms/step - loss: 0.7437 - accuracy: 0.7473 - val_loss: 0.7051 - val_accuracy: 0.7615
Epoch 6/10
1858/1858 [==============================] - 79s 43ms/step - loss: 0.6590 - accuracy: 0.7753 - val_loss: 0.6060 - val_accuracy: 0.7959
Epoch 7/10
1858/1858 [==============================] - 84s 45ms/step - loss: 0.5862 - accuracy: 0.7999 - val_loss: 0.4993 -

Testing the model

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_test_padded = pad_sequences(X_test, padding='post', dtype='float32')

score = model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {score[0]}, Test Accuracy: {score[1]}")

581/581 [==============================] - 15s 25ms/step - loss: 0.4056 - accuracy: 0.8601
Test Loss: 0.4055618345737457, Test Accuracy: 0.8600581288337708


Saving the model

In [ ]:
model.save('/content/drive/MyDrive/ISL/final_videos2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import joblib

# Assuming label_encoder is your LabelEncoder object

# Fit and transform your data with LabelEncoder if needed
# ...

# Save the LabelEncoder object to a file
joblib.dump(label_encoder, 'label_encoder2.joblib')



['label_encoder2.joblib']

In [ ]:
print(label_encoder.classes_)

['Birds' 'I dont know' 'address' 'angry' 'apple' 'bad' 'bathroom'
 'beautiful' 'body' 'bowl' 'bread' 'christmas' 'clock' 'die' 'dog' 'door'
 'eat' 'expensive' 'family' 'grow' 'handkerchief']
